In [82]:
import pandas as pd
import networkx as nx
import numpy as np
from fuzzywuzzy import fuzz

In [83]:
df_SC = pd.read_csv('SC_summary.csv', parse_dates=['start_date', 'end_date'])
df_SC.replace({pd.NaT: np.datetime64(800, 'M')}, inplace=True)

dates = [np.datetime64(592 + 3*i, 'M') for i in range(14)]

df_customer_cats = df_SC[["customer_SIC", "SIC_desc"]].drop_duplicates().sort_values(by='customer_SIC').rename(columns={"customer_SIC": "SIC", "SIC_desc": "SIC_desc"})
df_supplier_cats = df_SC[["supplier_SIC", "SIC_desc_s"]].drop_duplicates().sort_values(by='supplier_SIC').rename(columns={"supplier_SIC": "SIC", "SIC_desc_s": "SIC_desc"})

all_sic_codes = pd.concat([df_customer_cats, df_supplier_cats]).drop_duplicates().sort_values(by='SIC')

In [84]:
df_top_companies = pd.read_csv('topcompanys.csv')
df_top_companies

,Rank,Name,Symbol,marketcap,price (USD),country
0,1,Apple,AAPL,2714301890560,172.57,United States
1,2,Microsoft,MSFT,2297343246336,308.97,United States
2,3,Saudi Aramco,2222.SR,2175125589622,8.98,Saudi Arabia
3,4,Alphabet (Google),GOOG,1435133542400,117.92,United States
4,5,Amazon,AMZN,1131311792128,110.26,United States
...,...,...,...,...,...,...
7317,7318,Lukoil,LKOH.ME,0,56.31,Russia
7318,7319,Sberbank,SBER.ME,0,1.95,Russia
7319,7320,Gazprom,GAZP.ME,0,2.22,Russia
7320,7321,Ozon,OZON.ME,0,0.00,United States


In [88]:
all_customer_names = np.unique(df_SC["cusomter_name"])


def best_match(company):
    best_score = -1
    best_match = ''
    if company == 'Amazon':
        return 'Amazon.com, Inc.'
    if company == 'LVMH':
        return 'LVMH Moët Hennessy Louis Vuitton SE'
    if company == 'Visa':
        return 'Visa, Inc.'
    if company == 'TSMC':
        return 'Taiwan Semiconductor Manufacturing Co., Ltd.'
    if company == 'Tencent':
        return 'Tencent Holdings Ltd.'
    if company == 'Roche':
        return 'Roche Holding AG'
    if company == 'ICBC':
        return 'Industrial & Commercial Bank of China Ltd.'
    if company == 'ASML':
        return 'ASML Holding NV'
    if company == 'Hermès':
        return 'Hermès International SCA'
    if company == 'Costco':
        return 'Costco Wholesale Corp.'
    if company == 'Alibaba':
        return 'Alibaba Group Holding Ltd.'
    if company == 'Alphabet (Google)':
        return 'Alphabet, Inc.'
    if company == 'AMD':
        return "Advanced Micro Devices, Inc."
    if company == 'Toyota':
        return 'Toyota Motor Corp.'
    if company == 'Cisco':
        return 'Cisco Systems, Inc.'
    if company == 'Nike':
        return 'NIKE, Inc.'
    if company == 'Dior':
        return "Christian Dior SE"
    if company == 'Verizon':
        return "Verizon Communications, Inc."
    if company == 'CATL':
        return "Contemporary Amperex Technology Co., Ltd."
    if company == 'HSBC':
        return "HSBC Holdings Plc"
    if company == "Philip Morris":
        return "Philip Morris International, Inc."
    if company == 'BHP Group':
        return "BHP Group Ltd."
    if company == "Prosus":
        return "Naspers Ltd."
    if company == "China Life Insurance":
        return "New China Life Insurance Co., Ltd."
    if company == "Ping An Insurance":
        return "Ping An Insurance (Group) Co. of China Ltd."
    if company == "Honeywell":
        return "Honeywell International, Inc."
    if company == "CM Bank":
        return "China Merchants Bank Co., Ltd"
    if company == "Intel":
        return "Intel Corp."
    if company == 'Boeing':
        return "The Boeing Co."
    if company == "Keyence":
        return "KEYENCE Corp."
    if company == "AIA":
        return "AIA Group Ltd."
    if company == "Sony":
        return "Sony Group Corp."
    if company == "Porsche":
        return "Porsche Automobil Holding SE"
    if company == "IBM":
        return "International Business Machines Corp."
    if company == "Sinopec":
        return "Sinopec Shengli Oil Field Dynamic Group Co. Ltd."
    if company == "Mondelez":
        return "Mondelez International, Inc."
    if company =="Uber":
        return "Uber Technologies, Inc."
    if company == "HCA Healthcare":
        return "HCA Healthcare, Inc."
    if company == "Hindustan Unilever":
        return "Unilever Plc"
    if company == "GlaxoSmithKline":
        return "GSK Plc"
    if company == "Fast Retailing":
        return "FAST RETAILING CO., LTD."


    
    if company == 'Kweichow Moutai':
        return 'UNKNOWN'
    if company == 'International Holding Company':
        return 'UNKNOWN'
    if company == "PetroChina":
        return 'UNKNOWN'
    if company == "China Mobile":
        return 'UNKNOWN'
    if company == "China Construction Bank":
        return 'UNKNOWN'
    if company == "T-Mobile US":
        return 'UNKNOWN'
    if company == "Bank of China":
        return 'UNKNOWN'
    if company == "Tata Consultancy Services":
        return 'UNKNOWN'
    if company == "Inditex":
        return "UNKNOWN"
    if company == "LG Energy Solution":
        return "UNKNOWN"
    if company == "Wuliangye Yibin":
        return "UNKNOWN"	
    if company == "TAQA":
        return "UNKNOWN"
    if company == "China Shenhua Energy":
        return "UNKNOWN"
    if company == "China Yangtze Power":
        return "UNKNOWN"
    if company == "CNOOC":
        return "UNKNOWN"
    if company =="China Telecom":
        return "UNKNOWN"
    if company == "Petrobras":
        return "UNKNOWN"
    if company == "SABIC":
        return "UNKNOWN"
    if company == "Bank Central Asia":
        return "UNKNOWN"
    if company == "Walmex":
        return "UNKNOWN"


    for company_sc in all_customer_names:

        if fuzz.ratio(company, company_sc) > best_score:
            best_score = fuzz.ratio(company, company_sc)
            best_match = company_sc
    return best_match

In [89]:
names_top_200 = list(df_top_companies[df_top_companies['Rank']<=200]['Name'])
len(names_top_200)

200

In [90]:
df_translate_names = pd.DataFrame([[company, best_match(company)] for company in names_top_200], columns=['company', 'Factset name company'])
df_translate_names.to_csv('translation_top_200.csv')

In [112]:
translated_names_top_200 = list(df_translate_names['Factset name company'])
df_SC2 = df_SC[df_SC['cusomter_name'].isin(translated_names_top_200)]

In [113]:
len(df_SC2['cusomter_name'].value_counts())

177

In [94]:
def get_relevant_connections_for_date(df_SC, date):
    df_SC_date = df_SC.query('start_date <= @date <= end_date')
    return df_SC_date.reset_index().drop(columns=['index'])

def convert_df_to_graph(df):
    G = nx.MultiGraph()
    for row_nr, row in df.iterrows():
        start_date, end_date, customer_ID, cusomter_name, customer_SIC, SIC_desc, supplier_ID, supplier_name, supplier_SIC, SIC_desc_s = row
        G.add_node(customer_ID, name=cusomter_name, SIC = customer_SIC, desc=SIC_desc) # id=customer_SIC, desc=SIC_desc
        G.add_node(supplier_ID, name=supplier_name, SIC = supplier_SIC, desc=SIC_desc_s) #  id=supplier_SIC, desc=SIC_desc_s
        G.add_edge(supplier_ID, customer_ID, customer=customer_ID, customer_name = cusomter_name, supplier=supplier_ID, supplier_name=supplier_name, goods=supplier_SIC, descr=SIC_desc_s, edge_id=row_nr)
    return G

# stats on centrality, closeness etc -> see networkx


def get_suppliers(df, firmID):
    suppliers = df[df['customer_ID'] == firmID]
    if suppliers.empty:
        return suppliers, None
    
    sector_client_SIC = int(suppliers['customer_SIC'].median())
    if sector_client_SIC not in suppliers['customer_SIC'].values:
        sector_client_SIC = int(suppliers['customer_SIC'].max())
    return suppliers, sector_client_SIC

def get_clients(df, firmID):
    clients = df[df['supplier_ID'] == firmID]
    if clients.empty:
        return clients, None
    sector_supplier_SIC = int(clients['supplier_SIC'].median())
    if sector_client_SIC not in clients['supplier_SIC'].values:
        sector_client_SIC = int(clients['supplier_SIC'].max())
    return clients, sector_supplier_SIC


def score_dualsourcing(df, G):
    # scoring:
    # global model
    n= all_sic_codes.shape[0]
    list_sic_codes = list(all_sic_codes['SIC'].values)
    sectors_dualsourced = pd.DataFrame(np.zeros((n,n)), columns = list_sic_codes, index = list_sic_codes)
    sectors_not_dualsourced = pd.DataFrame(np.zeros((n,n)), columns = list_sic_codes, index = list_sic_codes)
    
    total_dual_sourcers = 0
    total_firms = 0
    for node in G.nodes:
        supplier_of_goods, SIC_code = get_suppliers(df, node)
        if supplier_of_goods.empty:
            continue

        suppliers_per_sector = supplier_of_goods['supplier_SIC'].value_counts()
        amnt_dualsourced_suppliers = sum(suppliers_per_sector>1)

        does_dual_sourcing = amnt_dualsourced_suppliers > 0

        if does_dual_sourcing:
            total_dual_sourcers += 1

        total_firms += 1

        sectors_that_were_dualsourced = list(suppliers_per_sector[suppliers_per_sector>1].index)
        sectors_that_were_not_dualsourced = list(suppliers_per_sector[suppliers_per_sector<=1].index)

        for supplier_sector_SIC in sectors_that_were_dualsourced:
            sectors_dualsourced[supplier_sector_SIC][SIC_code] += 1

        for supplier_sector_SIC in sectors_that_were_not_dualsourced:
            sectors_not_dualsourced[supplier_sector_SIC][SIC_code] += 1

    return sectors_dualsourced, sectors_not_dualsourced, total_dual_sourcers/total_firms


list_G = []
list_sectors_dualsourced = []
list_sectors_not_dualsourced = []
list_does_any_dualsourcing = []
for date in dates:
    df = get_relevant_connections_for_date(df_SC2, date)
    #df = df.head(25)
    G = convert_df_to_graph(df)
    nx.write_graphml(G, f'001_200_largest_{date}.graphml', prettyprint=True, edge_id_from_attribute='edge_id')
    sectors_dualsourced, sectors_not_dualsourced, pct_any_dualsource = score_dualsourcing(df, G)

    list_G.append(G)
    list_sectors_dualsourced.append(sectors_dualsourced)
    list_sectors_not_dualsourced.append(sectors_not_dualsourced)
    list_does_any_dualsourcing.append(pct_any_dualsource)

In [106]:
def sic_code_to_sector(sic_code):
    if sic_code < 1000:
        return "Agricultural"
    if sic_code < 1500:
        return "Mining"
    if sic_code<2000:
        return "Construction"
    if sic_code<3000:
        return "Consumer goods"
    if sic_code < 3500:
        return "Basic Industrial goods"
    if sic_code < 4000:
        return "Advanced industrial goods"
    if sic_code < 5000:
        return "Infrastructure"
    if sic_code < 6000:
        return "Retail"
    if sic_code < 7000:
        return "Financial"
    if sic_code < 8000:
        return "Services"
    if sic_code < 9000:
        return "Advanced services"
    else:
        return "Government"
    
all_sectors = ["Agricultural", "Mining", "Construction", "Consumer goods", "Basic Industrial goods", "Advanced industrial goods", "Infrastructure", "Retail", "Financial","Services", "Advanced services", "Government"]

In [107]:
def get_relevant_connections_for_date(df_SC, date):
    df_SC_date = df_SC.query('start_date <= @date <= end_date')
    return df_SC_date.reset_index().drop(columns=['index'])

def convert_df_to_graph(df):
    G = nx.MultiDiGraph()
    for row_nr, row in df.iterrows():
        start_date, end_date, customer_ID, cusomter_name, customer_SIC, SIC_desc, supplier_ID, supplier_name, supplier_SIC, SIC_desc_s = row
        G.add_node(customer_ID, name=cusomter_name, SIC = customer_SIC, desc=SIC_desc, global_sec = sic_code_to_sector(customer_SIC)) # id=customer_SIC, desc=SIC_desc
        G.add_node(supplier_ID, name=supplier_name, SIC = supplier_SIC, desc=SIC_desc_s, global_sec = sic_code_to_sector(supplier_SIC)) #  id=supplier_SIC, desc=SIC_desc_s
        G.add_edge(supplier_ID, customer_ID, customer=customer_ID, customer_name = cusomter_name, supplier=supplier_ID, supplier_name=supplier_name, goods=supplier_SIC, descr=SIC_desc_s, edge_id=row_nr)
    return G

# stats on centrality, closeness etc -> see networkx


def get_suppliers(df, firmID):
    suppliers = df[df['customer_ID'] == firmID]
    if suppliers.empty:
        return suppliers, None
    
    sector_client_SIC = int(suppliers['customer_SIC'].median())
    if sector_client_SIC not in suppliers['customer_SIC'].values:
        sector_client_SIC = int(suppliers['customer_SIC'].max())
    return suppliers, sector_client_SIC

def get_clients(df, firmID):
    clients = df[df['supplier_ID'] == firmID]
    if clients.empty:
        return clients, None
    sector_supplier_SIC = int(clients['supplier_SIC'].median())
    if sector_client_SIC not in clients['supplier_SIC'].values:
        sector_client_SIC = int(clients['supplier_SIC'].max())
    return clients, sector_supplier_SIC


def score_dualsourcing(df, G):
    # scoring:
    # global model
    n= len(all_sectors)
    list_sic_codes = list(all_sic_codes['SIC'].values)
    sectors_dualsourced = pd.DataFrame(np.zeros((n,n)), columns = all_sectors, index = all_sectors)
    sectors_not_dualsourced = pd.DataFrame(np.zeros((n,n)), columns = all_sectors, index = all_sectors)
    
    total_dual_sourcers = 0
    total_firms = 0

    percentage_dualsourcing = 0
    for node in G.nodes:
        supplier_of_goods, SIC_code = get_suppliers(df, node)
        if supplier_of_goods.empty:
            continue

        suppliers_per_sector = supplier_of_goods['supplier_SIC'].value_counts()
        amnt_dualsourced_suppliers = sum(suppliers_per_sector>1)

        does_dual_sourcing = amnt_dualsourced_suppliers > 0

        if does_dual_sourcing:
            total_dual_sourcers += 1

        total_firms += 1

        sectors_that_were_dualsourced = list(suppliers_per_sector[suppliers_per_sector>1].index)
        sectors_that_were_not_dualsourced = list(suppliers_per_sector[suppliers_per_sector<=1].index)

        for supplier_sector_SIC in sectors_that_were_dualsourced:
            sectors_dualsourced[sic_code_to_sector(supplier_sector_SIC)][sic_code_to_sector(SIC_code)] += 1

        for supplier_sector_SIC in sectors_that_were_not_dualsourced:
            sectors_not_dualsourced[sic_code_to_sector(supplier_sector_SIC)][sic_code_to_sector(SIC_code)] += 1

    return sectors_dualsourced, sectors_not_dualsourced, total_dual_sourcers/total_firms


list_G = []
list_sectors_dualsourced = []
list_sectors_not_dualsourced = []
list_does_any_dualsourcing = []
for date in dates:
    df = get_relevant_connections_for_date(df_SC2, date)
    #df = df.head(25)
    G = convert_df_to_graph(df)
    nx.write_graphml(G, f'001_200_largest_{date}.graphml', prettyprint=True, edge_id_from_attribute='edge_id')
    sectors_dualsourced, sectors_not_dualsourced, pct_any_dualsource = score_dualsourcing(df, G)

    list_G.append(G)
    list_sectors_dualsourced.append(sectors_dualsourced)
    list_sectors_not_dualsourced.append(sectors_not_dualsourced)
    list_does_any_dualsourcing.append(pct_any_dualsource)

In [95]:
list_pct_dualsourcing = [df.sum().sum() / (df.sum().sum() + df2.sum().sum()) for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]

In [98]:
abs_amnt_dual_sourcing = [df.sum().sum() for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]

In [101]:
import plotly.express as px

px.line(x=dates, y=list_pct_dualsourcing,
        labels={
            "x": "Date",
            "y": "Average dualsourcing per supplier sector",
        },
        title='Company-level view on suppliers: average dualsourcing per distinct supplier sector, top 200 companies by market cap')

In [100]:
px.line(x=dates, y=abs_amnt_dual_sourcing, title='Firmwise view on suppliers: Absolute amount of dual-sourced sectors')

In [102]:
fig = px.line(x=dates, y=[100*val for val in list_does_any_dualsourcing],
        labels={
            "x": "Date",
            "y": "percentage of companies utilising dualsourcing",
        },
        title='Firms that employ any kind of dual sourcing practices, top 200 companies by market cap')

fig.update_layout(yaxis_ticksuffix = "%")
fig.show()

In [108]:
list_results = [df.sum() / (df.sum() + df2.sum()) for df, df2 in zip(list_sectors_dualsourced, list_sectors_not_dualsourced)]

In [111]:

import plotly.graph_objects as go
fig = go.Figure()
anchos = [0.2] * 11

fig.add_trace(go.Bar(x=list_results[0].index, y=list_results[0], name=str(dates[0])))


fig.add_trace(go.Bar(x=list_results[8].index, y=list_results[8], name=str(dates[8])))

fig.add_trace(go.Bar(x=list_results[13].index, y=list_results[13], name=str(dates[13])))

fig.update_layout(title =  "Dual sourcing per supplier sector, top 200 companies by market cap", yaxis_title="Percentage dual sourcing")
fig

In [110]:
size_of_network = [len(G.nodes) for G in list_G]
px.line(x=dates, y=size_of_network,
        labels={
            "x": "Date",
            "y": "Companies in the supply chain",
        },
        title='Size of the Factset Supply Chain network')